In [158]:
import numpy as np
import cv2
np.random.seed(2)   # pour bloquer le random

# Définition des paramètres intrasèques de la caméra
  # focale
fx = 60
fy = 40
  # centre
cx = 0
cy = 0

A = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]]) # matrice intrasèque de la caméra (3*3)

# Génération de 3 angles aléatoire pour définir la rotation
rax,pitch,yaw = np.random.uniform(0,np.pi,size=3)

# Génération de la matrice de rotation

Rx = np.array([[1,0,0],[0, np.cos(rax), -np.sin(rax)],[0, np.sin(rax), np.cos(rax)]])
Ry = np.array([[np.cos(pitch), 0, np.sin(pitch)],[0,1,0],[-np.sin(pitch),0, np.cos(pitch)]])
Rz = np.array([[np.cos(yaw), -np.sin(yaw),0],[np.sin(yaw),np.cos(yaw),0],[0,0,1]])

R = Rz @ Ry @ Rx  # matrice de rotation (3*3)

# Génération de la translation de la caméra
T = np.random.uniform(-500,500,size=3)     # T = [tx,ty,tz]  (1*3)
T = T.reshape((3,1))                       # (3*1)

print(T)

[[ -64.67760738]
 [ -79.63219791]
 [-169.665179  ]]


In [159]:
# Projection d'un point 3D vers 2D
def projection3D2D(point3D,T,R,A) :
  # point 3D = [ Xw, Yw, Zw ]'   (1*3)
  # T : matrice de translation de la caméra : (3*1)
  # R : matrice de rotation : (3*3)
  # A : matrice intrasèque de la caméra : (3*3)

  PI = np.concatenate((np.eye(3),np.zeros((3,1))),axis=1)  # (3*4)

  Rt = np.concatenate((R,T),axis=1)               # (3*4)
  Rt = np.concatenate((Rt,np.array([[0,0,0,1]])),axis=0)   # (4*4)

  point3D_bis = np.concatenate((np.reshape(point3D,(3,1)),np.array([[1]])),axis=0)   #(4*1)

  point2D = A @ PI @ Rt @ point3D_bis   # point 2D = [u, v, w] (3*1)
  point2D = point2D / point2D[2]        # point 2D = [u, v, 1] (3*1)
  return point2D[:2]

pt3D = [[3],[1],[4]]
print("point 3D :\n ", pt3D)
pt2D = projection3D2D(pt3D,T,R,A)
print("point 2D :\n", pt2D)

point 3D :
  [[3], [1], [4]]
point 2D :
 [[23.44335066]
 [17.99596144]]


In [160]:
# Création des couples de point 2D,3D
def point3Daleatoire() :
  return np.array([[np.random.uniform(-500,500),np.random.uniform(-500,500),np.random.uniform(-500,500)]])

# points 3D
'''P1 = point3Daleatoire()     # (1*3) -> pour P3P
P2 = point3Daleatoire()
P3 = point3Daleatoire()
P4 = point3Daleatoire()'''
P1 = np.array([[-20,20,1]],dtype=np.float32)
P2 = np.array([[20,-20,1]],dtype=np.float32)
P3 = np.array([[-20,-20,1]],dtype=np.float32)
P4 = np.array([[20,20,1]],dtype=np.float32)

print(np.shape(P1))
print("P1\n",P1)
points3D = np.concatenate((P1,P2,P3),axis=0);     # (3*3)
print("pt3D\n",points3D)

# point 2D
p1 = projection3D2D(P1,T,R,A)   # (2*1)
p1 = np.reshape(p1,(1,2))       # (1,2)
p2 = projection3D2D(P2,T,R,A)
p2 = np.reshape(p2,(1,2))
p3 = projection3D2D(P3,T,R,A)
p3 = np.reshape(p3,(1,2))
p4 = projection3D2D(P4,T,R,A)
p4 = np.reshape(p4,(1,2))


points2D = np.concatenate((p1,p2,p3),axis=0);    # (3*2)
print("p1\n",p1)
print("pt2D\n",points2D)
print(np.shape(points2D))
# Application de solveP3P pour retrouver la position de la caméra
retval, rvec, tvecs =  cv2.solveP3P(points3D,points2D,A,None, flags = cv2.SOLVEPNP_P3P)

rvec = np.array(rvec)
tvecs = np.array(tvecs)

print("rvec taille",np.shape(rvec))
print(np.shape(tvecs))
print(type(rvec))
print(type(tvecs))

print(rvec)


(1, 3)
P1
 [[-20.  20.   1.]]
pt3D
 [[-20.  20.   1.]
 [ 20. -20.   1.]
 [-20. -20.   1.]]
p1
 [[19.82338197 14.04197347]]
pt2D
 [[19.82338197 14.04197347]
 [25.30980495 22.56202743]
 [13.37249182 20.62929912]]
(3, 2)
rvec taille (2, 3, 1)
(2, 3, 1)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[[ 1.77939346]
  [ 0.33101425]
  [-1.16312262]]

 [[-2.67676642]
  [-0.46680933]
  [-0.13303   ]]]


In [161]:
# Calcul erreur estimation :
def erreur_estimation(pt, pt_estimation):
    erreur = 0
    for i in range(len(pt)):
      erreur += (pt[i] - pt_estimation[i])**2
    return np.sqrt(erreur)

In [162]:


# Mesure de l'erreur d'estimation
nb_solutions = len(rvec)
print("nombre de solutions trouvées =", nb_solutions)

for i in range (nb_solutions):
  # Récupération des matrices de rotation et translation

  rodriguez = rvec[i]     # (3*1)
  print("taille rodirguez =",np.shape(rodriguez))
  R_P3P = cv2.Rodrigues(rodriguez)[0]    # matrice de rotation : (3*3)
  print("R_P3P=",R_P3P,"\n")
  print("R",R,"\n")

  T_P3P = tvecs[i]    # matrice de translation : (1*3)
  T_P3P = np.reshape(T_P3P,(3,1))    # (3*1)
  print("T=",T_P3P,"\n")


  # Estimation des points
  p1_P3P = np.reshape(projection3D2D(P1,T_P3P,R_P3P,A),(1,2))

  p2_P3P = np.reshape(projection3D2D(P2,T_P3P,R_P3P,A),(1,2))
  p3_P3P = np.reshape(projection3D2D(P3,T_P3P,R_P3P,A),(1,2))
  p4_P3P = np.reshape(projection3D2D(P4,T_P3P,R_P3P,A),(1,2))

  # Création d'un tableau des points estimés
  pt_2D_P3P = np.concatenate((p1_P3P,p2_P3P,p3_P3P,p4_P3P),axis=0)    # (4,2)
  print("pt_2D_P3P = ",pt_2D_P3P,"\n")

  # Création d'un tableau des points originaux

  pt_2D = np.concatenate((p1,p2,p3,p4),axis=0)    # (4,2)
  print("pt_2D = ",pt_2D)

  # Calcul de l'erreur d'estimation

  for i in range(len(pt_2D)):
    erreur = erreur_estimation(pt_2D[i],pt_2D_P3P[i])
    print("erreur = ",erreur)





nombre de solutions trouvées = 2
taille rodirguez = (3, 1)
R_P3P= [[ 0.51073426  0.64908145 -0.56377636]
 [-0.25496915 -0.51189907 -0.82033534]
 [-0.82106105  0.56271895 -0.09594865]] 

R [[-0.93911024 -0.3429191   0.02187359]
 [-0.30293508  0.85629405  0.41831907]
 [-0.16217982  0.38622144 -0.90803673]] 

T= [[ 62.16514689]
 [ 74.35227344]
 [167.24577978]] 

pt_2D_P3P =  [[19.82338153 14.04197376]
 [25.30980513 22.56202762]
 [13.37249227 20.62929862]
 [31.4098481  14.37053949]] 

pt_2D =  [[19.82338197 14.04197347]
 [25.30980495 22.56202743]
 [13.37249182 20.62929912]
 [32.61907091 16.41175791]]
erreur =  5.303164592503119e-07
erreur =  2.6286598580736774e-07
erreur =  6.751226459097423e-07
erreur =  2.3725076373768688
taille rodirguez = (3, 1)
R_P3P= [[ 0.93911022  0.34291913  0.02187372]
 [ 0.30293506 -0.85629403  0.41831912]
 [ 0.16217997 -0.38622144 -0.9080367 ]] 

R [[-0.93911024 -0.3429191   0.02187359]
 [-0.30293508  0.85629405  0.41831907]
 [-0.16217982  0.38622144 -0.90803673